# **Model Server**

_______________________________________________________________________________

<a id="top"></a>
## **steps**
**[nuclio code section](#nuclio)**<br>
    - [inference server](#server)<br>
**[deploy](#deploy)**<br>
**[test deployment](#test)**<br>
**[test saved model object](#testingoutside)**


<a id="nuclio"></a>
_______________________________________________________________________________

## **nuclio code section**

In [ ]:
# nuclio: ignore
import nuclio

Install the following packages so they are available to the function:

In [ ]:
%%nuclio cmd
pip install -U -q git+https://github.com/mlrun/mlrun.git@development
pip install -U -q kfserving
pip install -U -q numpy==1.17.4 
pip install -U -q tensorflow==2.0.0b1
pip install -U -q pandas
pip install -U -q azure
pip install -U -q git+https://github.com/yjb-ds/functions-demo.git

In [1]:
import warnings
warnings.simplefilter(action="ignore", category=FutureWarning)

import kfserving
import numpy as np
from pickle import load
from typing import List
from tensorflow.keras.models import Sequential

In [2]:
TARGET_PATH = '/User/mlrun/simdata'
MODEL_NAME = 'model.pkl'

In [3]:
class MyKerasClassifier(kfserving.KFModel):
    def __init__(self,
                 name: str,
                 model_dir: str,
                 classifier = None):
        """TFKerasClassifier
        
        Server model wrapper.
        
        :param name:            model name
        :param model:       path of stored model
        :param classifier:      class type of classifier model
        
        """
        super().__init__(name)
        self.name = name
        self.model_dir = model_dir
        if classifier:
            self.ready = True

    def load(self):
        """Load model from storage.
        
        Note that loading to load a model we specify only a folder. Our custom
        loader takes care of the details.
        """
        model_file = os.path.join(
            kfserving.Storage.download(self.model_dir), MODEL_NAME)
        self.classifier = load(model_file)
    
    def predict(self, body):
        """Generate model predictions from sample.
        
        :param body: A list of observations, each of which is an 1-dimensional feature vector.
            
        Returns model predictions as a `List`, one for each row in the `body` input `List`.
        """
        try:
            feats = np.asarray(body['instances'])
            result: np.ndarray = self.classifier.predict(feats)
            return result.tolist()
        except Exception as e:
            raise Exception(f"Failed to predict {e}")

In [4]:
# nuclio: end-code

<a id="deploy"></a>
_______________________________________________________________________________

## **deploy**

In [14]:
from mlrun import new_model_server, mount_v3io

In [15]:
TARGET_PATH = "/User/mlrun/simdata"
MODEL_NAME = "model"

In [16]:
fn = new_model_server('tfkeras_inference_model', 
                      models={"tfkeras_pickle": TARGET_PATH}, 
                      model_class="MyKerasClassifier").apply(mount_v3io())

While debugging your project code, you may want to set the following flag to `True` in order to ensure that all the layers in your image get rebuilt and your changes included:

In [17]:
fn.spec.no_cache=True

After running the following cell, you can start watching the function's deployment by selecting **Projects** in the Iguazio platform UI, and clicking on the project name, and selecting the correct function and version:

In [18]:
addr = fn.deploy(project=("github-demos"))

[mlrun] 2020-01-13 15:37:32,318 deploy started
[nuclio] 2020-01-13 15:43:41,700 (info) Build complete
[nuclio] 2020-01-13 15:43:51,213 (info) Function deploy complete
[nuclio] 2020-01-13 15:43:51,218 done creating tfkeras-inference-model, function address: 3.135.246.153:31327


<a id="deploy"></a>
_______________________________________________________________________________

## **test**

In [ ]:
import pandas as pd
import requests
import json

#### grab 2 rows of data - balanced dataset

In [ ]:
features = pd.read_csv("x_test_50.csv").iloc[:2,:]
labels = pd.read_csv("y_test_50.csv").iloc[:2,:]

In [ ]:
features

In [ ]:
labels

#### create an event and wrap it in json

In [ ]:
event = {"instances" : features.values.tolist()}

If the notebook is restarted for some reason, however the function has already been deployed, simply uncomment the following cell and paste in the original endpoint here.  You can 
retrieve the function's enpoint address from the platform ui under **Projects**. Look for
the project name you gave to the deployment of interest, in our case **mlrun-demos**.
Click on the function in the project, copy the 'Invocation URL' and paste here:

In [ ]:
# addr = "3.137.70.243:31811"

In [ ]:
resp = requests.post(addr + "/tfkeras_pickle/predict", json=event)

In [ ]:
json.loads(resp.content)

### Test of Estimated Model Object 

Here we simply grab the estimated model file created on the **[kubeflow pipeline](kubeflow%20pipeline.ipynb)**, load it, and run a test matrix through it.

In [ ]:
import joblib
import os
model_file = os.path.join(TARGET_PATH, MODEL_NAME)
keras_model = joblib.load(open(model_file, 'rb'))

In [ ]:
testvals = np.asarray(features.values)

In [ ]:
keras_model.predict(testvals)